#Install Lib

In [3]:
# Installl Db2
!pip install ibm_db
# Install Spacy to NER_EN
'''
!pip install spacy
# Install Spacy to NER_VI
!pip install underthesea
# Install Spacy to NER_VI
!pip3 install vncorenlp
!git clone https://github.com/vncorenlp/VnCoreNLP.git
'''

     |████████████████████████████████| 727kB 2.8MB/s 
  Created wheel for ibm-db: filename=ibm_db-3.0.2-cp36-cp36m-linux_x86_64.whl size=38977341 sha256=77f338e57efc11cf36adb4e6ebcc7119a1db9b101a8a38f07e6a3732c07f8e0f
  Stored in directory: /root/.cache/pip/wheels/84/64/42/10b217b545e8085b72160632c61bfa0909f661dad89326a956
Successfully built ibm-db


'\n!pip install spacy\n# Install Spacy to NER_VI\n!pip install underthesea\n# Install Spacy to NER_VI\n!pip3 install vncorenlp\n!git clone https://github.com/vncorenlp/VnCoreNLP.git\n'

#Connect Database and import lib

In [4]:
from vncorenlp import VnCoreNLP
from underthesea import ner
import ibm_db

In [5]:
conn = ibm_db.connect("DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=xbr09096;PWD=q3p+g7szbpg06pm7","","")

#Get comments from Database

In [ ]:
def get_value_from_database(table_name, index):
  selectStmt = ibm_db.exec_immediate(conn,"select * from " + table_name)
  list_comment = []
  while ibm_db.fetch_row(selectStmt):
    text = ibm_db.result(selectStmt,index)
    list_comment.append(text.replace('.',' . '))
  return list_comment

In [ ]:
# Table Reviews_Book_VI or Reviews_Book_EN, index 5: Review_Content
# Table List_comments_VI or List_comments_VI, index 1: Comments_Content
list_comment_VI = get_value_from_database('Reviews_Book_VI',5) + get_value_from_database('List_comments_VI',1)
list_comment_EN = get_value_from_database('Reviews_Book_EN',5) + get_value_from_database('List_comments_EN',1)

#Name entity recognition with underthesea and VnCoreNLP library (BERT_VI)

In [ ]:
def named_entity_recognition_VI(list_sentence):
  name = []
  for sentence in list_sentence:
    ners = ner(sentence)
    for i in range((len(ners))):
      if ners[i][1] == 'Np':
        name.append(ners[i][0])

    try:
      ners = annotator.ner(sentence)
    except:      
      print("load_VnCoreNLP")
      # To perform word segmentation, POS tagging and then NER
      annotator = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g')
      ners = annotator.ner(sentence)

    for i in ners:
      for j in i:
        if j[1] != 'O':
          name.append(j[0].replace('_',' '))
  
  return set(name)

test_sentences = ['Tác phẩm của bác Nguyễn Nhật Ánh thật có giá trị nhân văn! Nó Nói về quê hương Quảng Nam của mình thật đẹp!','Anh Ngạn trong "Mắt biếc" đúng là một kẻ si tình, cứ mãi yêu hà lan']
print(named_entity_recognition_VI(test_sentences))

load_VnCoreNLP
{'Quảng Nam', 'Nguyễn Nhật Ánh', 'hà lan', 'Ngạn'}


In [ ]:
list_comment_ner_VI = named_entity_recognition_VI(list_comment_VI)

load_VnCoreNLP


#Name entity recognition using Spacy library (BERT_EN)

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
key = []
for i in list_comment_EN:
  doc = nlp(i)
  for entity in doc.ents:
    key.append(entity.label_)

key = set(key)
key

{'CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERSON',
 'PRODUCT',
 'TIME',
 'WORK_OF_ART'}

In [ ]:
df = {}
for i in key:
  df[i] = []

for i in list_comment_EN:
  doc = nlp(i)
  for entity in doc.ents:
    df[entity.label_].append(entity.text)

In [ ]:
df

In [ ]:
values = []
for key, value in df.items():
  print(key,value)

PRODUCT ['REAL', 'vật và', "Phuc'grandfather", 'Besides', 'Nhi - one', 'lớn cũng', 'cần những câu', 'lớn cho trọn vẹn', 'lớn cho trọn', 'Lucky Luke', 'sao lun']
ORG ['Nguyễn Nhật Ánh', 'Nguyễn Nhật Ánh', 'Inner Child', 'TNA', 'TNA', 'Anyways', 'SIMPLE', 'Clothes', 'Nguyen Nhat Anh', 'Ngạn', 'Nguyễn Nhật Ánh', 'Trà', 'Nguyễn Nhật Ánh', 'Ngạn', 'Nguyen Nhat Anh', 'Ngạn', 'Nguyễn Nhật Ánh', 'Recommend', 'Sad Simple Romance', 'Buồn', 'Thảm', 'Nguyan Nhat Anh', 'Có', 'lời', 'của lòng', 'dư vị ngọt', 'Childhood', 'Cuội', 'NẮNG MƯA LÀ BỆNH CỦA TRỜITƯƠNG', 'Uhm', 'Nguyễn Nhật Ánh', 'Sài Gòn', 'Nguyễn Nhật Ánh', 'Miền', "Nguyễn Nhật Ánh's", 'Ngay Xua Co', 'Vinh ( Vinh Com', 'Everyone', 'Phuc, Vinh', 'Nguyễn Nhật Ánh', 'Tuổi', 'Anh', 'Anh', 'Dang & Tham', 'Betrayal', 'Nguyen Nhat Anh', 'Nice', 'humourous!it', 'Anh', 'Anh', 'Mui, Ti, Hai', 'Hiep', 'Hơi', 'un un gô', 'buồn', 'câu', 'ngày', 'được đưa vào', 'Nguyễn Nhật Ánh', 'chính người', 'realize1', 'Characteristics', 'N . N', 'là siêu chợ', 'Ngu

In [ ]:
# ORG, GPE, PERSON, LAW, NORP
list_comment_ner_EN = []
for key, value in df.items():
  if key in ('ORG', 'GPE', 'PERSON', 'LAW', 'NORP'):
    for i in value:
      list_comment_ner_EN.append(i)

list_comment_ner_EN = set(list_comment_ner_EN)
list_comment_ner_EN

In [ ]:
column_name = ['VI','EN']
#export csv
pd.DataFrame((list_comment_ner_VI,list_comment_ner_EN),index=column_name).to_csv('name.csv')

Note: Open file `name.csv` and manually delete wrong words

#Save to database 

In [24]:
repair_file = open('./name.csv').read()
repair_file = repair_file.split(',')
repair_file = set(repair_file)

In [36]:
try:
  dropStmt = ibm_db.exec_immediate(conn,'drop table Name')
except:
  print('Not exist')
createStmt = ibm_db.exec_immediate(conn,'create table Name(name_value clob)')

In [37]:
insertSQL = "insert into Name(name_value) values ('{}')"
new_file = []
for i in repair_file:
  if i != '':
    insertStmt = ibm_db.exec_immediate(conn,insertSQL.format(i.replace("'","''")))

In [43]:
selectStmt = ibm_db.exec_immediate(conn,'select * from Name')

name_value = {'name':[]}
while ibm_db.fetch_row(selectStmt):
  name_value['name'].append(ibm_db.result(selectStmt,0))

In [46]:
import pandas as pd

ex = pd.DataFrame(name_value).to_csv('name.csv',index=False)